In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
train_data = sqlContext.read.load('/FileStore/tables/train.csv',format='com.databricks.spark.csv',
                                  header='true',
                                  inferSchema='true')
pd.DataFrame(train_data.take(3), columns = train_data.columns)

In [3]:
#Basic Statistics
train_data.describe().toPandas().transpose()

In [4]:
from pyspark.sql.functions import isnan, when, count, col

train_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_data.columns]).show()

In [5]:
test_data = sqlContext.read.load('/FileStore/tables/test.csv',format='com.databricks.spark.csv',
                                  header='true',
                                  inferSchema='true')
pd.DataFrame(test_data.take(3), columns = test_data.columns)

In [6]:
test_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in test_data.columns]).show()

In [7]:
avgAge = train_data.select("Age").unionAll(test_data.select("Age"))
agecount = avgAge.count()

In [8]:
train_data.count() + test_data.count()

In [9]:
from pyspark.sql.functions import col, avg
avgAge.agg(avg(col("Age"))).show()

In [10]:
df = train_data.fillna({"Age" : 30})
df = df.fillna({"Embarked" : "S"})

In [11]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [12]:
df1 = test_data.fillna({"Age" : 30})

In [13]:
df1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df1.columns]).show()

In [14]:
df = train_data.unionAll(test_data)

In [15]:
df.count()

In [16]:
from pyspark.sql.functions import split
split_col = split(df['Name'], ',')
df_name = df.withColumn('Title', split_col.getItem(1))
#df = df.withColumn('NAME2', split_col.getItem(1))
df_name.show()

In [17]:
from pyspark.sql.functions import *
newDf1 = df_name.withColumn('Title', regexp_replace('Title', ' ', '   '))

In [18]:
newDf1.show()

In [19]:
split_col = split(newDf1['Title'], ' ')
newDf1 = newDf1.withColumn('Title', split_col.getItem(3))
newDf1.show()

In [20]:
newDf = newDf1.withColumn('Title', regexp_replace('Title', 'Miss.', 'Miss'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mrs.', 'M_rs'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Ms.', 'Miss'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mme.', 'Miss'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mlle.', 'Miss'))
#newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mrs.', 'Miss'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Dr.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Major.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Lady.', 'Mrs'))
#newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mrs.', 'Miss'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Countess.', 'Mrs'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Don.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Sir.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Capt.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Col.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Rev.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Jonkheer.', 'Mr'))
newDf = newDf.withColumn('Title', regexp_replace('Title', 'Mr.', 'Mr'))
df = newDf.withColumn('Title', regexp_replace('Title', 'Master.', 'Master'))
df.show()

In [21]:
df = df.withColumn('Title', regexp_replace('Title', 'the', 'M_rs'))

In [22]:
X=df.select(df.Title).distinct().show()

In [23]:
df.printSchema()

In [24]:
df0 = df.select(df.Pclass, df.Sex, df.Age, df.SibSp, df.Parch, df.Fare, df.Embarked, df.Title)

In [25]:
df0.printSchema()

In [26]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ['Pclass','Sex','Embarked']
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [27]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "Survived", outputCol = "label")
stages += [label_stringIdx]

In [28]:
# Transform all features into a vector using VectorAssembler
numericCols = ["Age", "SibSp", "Parch", "Fare"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [29]:
cols = train_data.columns
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(train_data)
dataset0 = pipelineModel.transform(train_data)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset0 = dataset0.select(selectedcols)
#display(dataset)
#type(dataset)
dataset0.show()

In [30]:
dataset0.show()